In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

In [ ]:
!kaggle datasets download -d thanatoz/flower-classification-dataset

In [ ]:
import zipfile
import os

# Specify the path to your zip file
zip_file_path = "/content/flower-classification-dataset.zip"

# Specify the target directory for extraction
extracted_dir = "/content/flower-classification-dataset"

# Create the target directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# List the files in the extracted directory
extracted_files = os.listdir(extracted_dir)
print("Extracted files:", extracted_files)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm

In [ ]:
ROOT_PATH='/content/flower-classification-dataset'
df=pd.read_csv(os.path.join(ROOT_PATH,'labels.csv'))

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
plt.figure(figsize=(20,6))
df.category.value_counts().plot(kind='bar')

In [ ]:
s=df.category.value_counts()
s.index[:5], s.values[:5]
print('The highest class is','[',s.index[0],']','with','[',s.values[0],']','images')
print('The lowest class is','[',s.index[-1],']','with','[',s.values[-1],']','images')


In [ ]:
unique_df=df.iloc[df.category.drop_duplicates().index].sort_values('category').reset_index(drop=True)

In [ ]:
unique_df

In [ ]:
fig,axis=plt.subplots(34,3,figsize=(15,225))
count=-1
for i,j in enumerate(unique_df.values):
  if i%3==0:
    count+=1
  axis[count,i%3].imshow(Image.open(os.path.join(ROOT_PATH,'files',f'{j[0]}.jpg')).resize((200,200)))
  axis[count,i%3].set_title(str(j[1]))

In [ ]:
target_class=51
fig,axis=plt.subplots(34,3,figsize=(15,225))
count=-1
for i,j in enumerate(df[df.category==target_class].head(102).values):
  if i%3==0:
    count+=1
  axis[count,i%3].imshow(Image.open(os.path.join(ROOT_PATH,'files',f'{j[0]}.jpg')).resize((200,200)))
  axis[count,i%3].set_title(str(j[1]))

In [ ]:
X=[]
y=[]

for image,label in tqdm(zip(df.image_id.values,df.category.values),total=len(df)):
  try:
    Xt=np.array(Image.open(os.path.join(ROOT_PATH,'files',f'{image}.jpg')).resize((128,128)))
    yt=label
    X.append(Xt)
    y.append(yt)
  except:
    print(os.path.join(ROOT_PATH,'files',f'{image}.jpg'))

X = np.array(X)
y = np.array(y)

X.shape,y.shape

In [ ]:
files = os.listdir(os.path.join(ROOT_PATH,'files'))
print('The number of total images is:',len(files))
print('The total number of labels in labels.csv is:',len(df))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=0.2)

In [ ]:
from keras.utils import to_categorical

In [ ]:
train_y=to_categorical(train_y)
test_y=to_categorical(test_y)

train_y.shape,test_y.shape

In [ ]:
train_X.shape,test_X.shape

In [ ]:
plt.figure(figsize=(14,14))
for i in range(12):
  plt.subplot(4,3,i+1)
  plt.imshow(train_X[i])

plt.show()

In [ ]:
plt.figure(figsize=(18,6))
df.category.value_counts().plot(kind='bar')

In [ ]:
height=[]
width=[]
for image in tqdm(os.listdir(os.path.join(ROOT_PATH,'files'))):
  ht,wt=Image.open(os.path.join(ROOT_PATH,'files',f'{image}')).size
  height.append(ht)
  width.append(wt)

In [ ]:
from collections import Counter

In [ ]:
Counter(height),Counter(width)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
!rm -rf preview
!mkdir preview

In [ ]:
datagen=ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
img=load_img(os.path.join(ROOT_PATH,'files','0.jpg'))
x=img_to_array(img)
x=x.reshape((1,)+ x.shape)

In [ ]:
i=0
for batch in datagen.flow(x,batch_size=1,save_to_dir='preview',save_prefix='f',save_format='jpg'):
  i+=1
  if i>20:
    break


In [ ]:
x=[]
for image in os.listdir('preview'):
  xt=np.array(Image.open(os.path.join('preview',image)).resize((128,128)))
  x.append(xt)

In [ ]:
plt.figure(figsize=(14,14))
for i in range(12):
  plt.subplot(4,3,i+1)
  plt.imshow(x[i])

plt.show()

In [ ]:
from keras.layers import MaxPooling2D, Conv2D, Activation,Flatten, Dense, Dropout
from keras.models import Sequential

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(128,128,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(103))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size=16

train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=train_datagen.flow(
    train_X,
    train_y,
    batch_size=batch_size,
    shuffle=True)

valid_generator=test_datagen.flow(
    test_X,
    test_y,
    shuffle=False)

In [ ]:
hist=model.fit_generator(
    train_generator,
    epochs=50,
    validation_data=valid_generator)